In [17]:
import os 
import json
import pickle
from datetime import datetime
from copy import deepcopy

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dropout, Bidirectional, Attention, Concatenate, Dense
from keras.callbacks import ModelCheckpoint

from konlpy.tag import Okt

from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats("png2x") # svg, retina, png2x ...
mpl.style.use("seaborn-v0_8")
mpl.rcParams.update({"figure.constrained_layout.use": True})
sns.set_context("paper") 
sns.set_palette("Set2") 
sns.set_style("whitegrid") 

plt.rc("font", family = "Malgun Gothic")
plt.rcParams["axes.unicode_minus"] = False

# 파일 저장시 파일명의 용이성
def now_time():
    now = datetime.now()
    return now.strftime('%Y%m%d_%H_%M_%S')

### 불러오기

In [18]:
base_dir_1 = 'data_prep/data/_2_after_prep/corpus_method_1/'


with open(base_dir_1 + "sentences.pkl","rb") as f:
    sentences = pickle.load(f)

with open(base_dir_1 + "labels.pkl","rb") as f:
    labels = pickle.load(f)

with open(base_dir_1 + "sentences_corpus_word_index.json","r") as f:
    sentences_corpus_word_index = json.load(f)

with open(base_dir_1 + "label_corpus_word_index.json","r") as f:
    label_corpus_word_index = json.load(f)

---

### 패딩 작업

In [19]:
# 이렇게 분리 하는 이유 (기존의 트레인 테스트의 tsv를 하나로 합쳤기 때문)
train_sentences, test_sentences = sentences[:15005], sentences[15005:] 
train_labels, test_labels = labels[:15005], labels[15005:]

In [20]:
decoder_input_data = deepcopy(train_labels) # 디코더의 입력 : end 토큰만 제외
target_sequences   = deepcopy(train_labels)   # 타겟 데이터 : 에서 start 토큰을 제외한 값

In [21]:
# 정답 라벨에 각 요소별 시작 1 토큰 추가 및 끝토큰 추가
for i in decoder_input_data:
    i.insert(0,1)
for i in target_sequences:
    i.append(2)

In [22]:
print(decoder_input_data[0])
print(target_sequences[0])

[1, 3]
[3, 2]


- 코퍼스 인덱스 : 단어 역변환 (나중에 찾기 쉽게하기 위함)

In [23]:
sentences_corpus_index_word = {sentences_corpus_word_index[key]:key for key in sentences_corpus_word_index}
label_corpus_index_word = {label_corpus_word_index[key]:key for key in label_corpus_word_index}

- 패딩 작업

In [24]:
max_text_len = max([len(i) for i in train_sentences])
text_padded = pad_sequences(train_sentences, maxlen=max_text_len, padding='post')

max_label_len = max([len(i) for i in decoder_input_data])
decoder_input_data_padded = pad_sequences(decoder_input_data, maxlen=max_label_len, padding='post')
target_sequences_padded = pad_sequences(target_sequences, maxlen=max_label_len, padding='post')

- 전체 코퍼스 크기

In [25]:
corpus_size = len(sentences_corpus_word_index)
corpus_size_label = len(label_corpus_word_index)
corpus_size, corpus_size_label

(38712, 14)

In [26]:
# 불러온 데이터 확인하기
for i in sentences[256]:
    print(sentences_corpus_index_word[i], end=' ')

print("")
for i in labels[256]:
    print(label_corpus_index_word[i])

적 화 다 된거 냐 ? ㅋㅋㅋㅋㅋㅋ 기 가 차다 , 빨갱이 들 이 많긴 많은 가봐 이나라 에 
기타 혐오


In [27]:
corpus_size_label

14

In [28]:
import model._1_lstm_with_attention_model as model_lstm_att

In [29]:
model, encoder_model, decoder_model = \
model_lstm_att.seq2seq_with_attention(max_text_len, corpus_size, max_label_len, corpus_size_label)

In [30]:
# _ = load_model('checkpoint_model1\epoch_0002_metrics_0.1557,0.8384,0.1929,0.7882.h5')
# model.set_weights(_.get_weights())

In [ ]:
# # 모델 학습
# check_path = 'checkpoint_'
# check_path_list = os.listdir(check_path)
# # 학습시
# checkpoint = ModelCheckpoint(check_path + f'/epoch_'+'{epoch:04d}_metrics_{loss:.4f},{accuracy:.4f},{val_loss:.4f},{val_accuracy:.4f}.h5', save_freq='epoch')
# 모델 학습
history_model = model.fit([text_padded, decoder_input_data_padded], target_sequences_padded,
                        validation_split=0.1,
                        epochs=10,
                        # initial_epoch=len(check_path_list),
                        batch_size=64,
                        # callbacks=[checkpoint],
                        verbose=1)

In [33]:
sets_for_predict = sentences_corpus_word_index,max_text_len, label_corpus_word_index, label_corpus_index_word, max_label_len
moonjang_ = '안녕 하십니까 반가워요?'
model_lstm_att.predict_from_seq(moonjang_, encoder_model, decoder_model, sets_for_predict)

['clean']

### 테스트 데이터를 이용하여 정답 확인

In [34]:
from tqdm import trange
from IPython.display import display, clear_output

In [35]:
correct = 0
has_more_than_one = 0
sum_pred = 0
predict_labels=[]
correct_labels=[]
for i in range(len(test_sentences)): #len(test_sentences)):
    test_sentences[i], test_labels[i]
    test_sen = ''
    for j in test_sentences[i]:
        test_sen += sentences_corpus_index_word[j] + ' '
    predict_label = model_lstm_att.predict_from_seq(test_sen, encoder_model, decoder_model,sets_for_predict)
    correct_label = [label_corpus_index_word[k] for k in test_labels[i]]
    sum_pred += 1
    correct += 1 if predict_label == correct_label else 0
    has_more_than_one += 1 if set(predict_label).intersection(set(correct_label)) != set() else 0

    predict_labels.append(predict_label)
    correct_labels.append(correct_label)

    # print(f'''
    # 문제 : {test_sen}
    # 정답 : {correct_label} | 예측 : {predict_label}
    # {('정답!' if predict_label == correct_label else '오답!')}
    # ''')

    clear_output(wait=True)
    print(f'현재 인덱스 진행 ({i}/{len(test_sentences)})')
    print('정답률:',correct / sum_pred * 100)
print('하나라도 겹치는것이 있는 비율:',has_more_than_one / sum_pred * 100)

현재 인덱스 진행 (3736/3737)
정답률: 61.33261974846134
하나라도 겹치는것이 있는 비율: 67.995718490768


### 틀린 문장 확인해보기

In [ ]:
# 테스트 데이터 number 까지의
number = 10
print("완전 정확히 못맞춘 문장들")
for s,c,p in zip(test_sentences[:number],correct_labels[:number],predict_labels[:number]):
    if c != p:
        sen = ''
        for w in s:
            sen += sentences_corpus_index_word[w] + ' '
        print("문장: ",sen)
        print("정답: ",c,",예측: ",p)
        print(" ")
        
print("-"*100)
print("하나도 못 맞춘 문장들")
for s,c,p in zip(test_sentences[:number],correct_labels[:number],predict_labels[:number]):
    if c != p:
        sen = ''
        for w in s:
            sen += sentences_corpus_index_word[w] + ' '
        if set(c).intersection(set(p)) == set():
            print("문장: ",sen)
            print("정답: ",c,",예측: ",p)
            print(" ")

print("-"*100)
print("하나라도 맞춘 문장들")
for s,c,p in zip(test_sentences[:number],correct_labels[:number],predict_labels[:number]):
    if c != p:
        sen = ''
        for w in s:
            sen += sentences_corpus_index_word[w] + ' '
        if set(c).intersection(set(p)) != set():
            print("문장: ",sen)
            print("정답: ",c,",예측: ",p)
            print(" ")


문장:  전라도 랑 조선족 페미 합치 면 30 퍼쫌 되지 않노 
정답:  ['여성/가족', '인종/국적', '지역'] ,예측:  ['여성/가족', '지역']
 
문장:  다문화 난 찬성 함 한 녀 들 이 안 대주니깐 그거 라도 찬성 해야 됨 
정답:  ['여성/가족', '인종/국적'] ,예측:  ['여성/가족']
 
문장:  엥 ? 남자 30 대 함소원 50 대 되면 왜 남자 가 함소원 을 버리노 ? 상폐남 을 버리고 더 어린 소 남 으로 갈아타는거지 
정답:  ['연령'] ,예측:  ['남성', '연령']
 
문장:  그래도 지 잡 ㄷㅐ 나온 년 보다는 이대 나온 년 따먹고 싶다 이기 
정답:  ['여성/가족', '기타 혐오'] ,예측:  ['여성/가족']
 
문장:  30~40 대 김치 년 이 많이 있어서 그런 거지 
정답:  ['여성/가족', '연령'] ,예측:  ['여성/가족']
 
문장:  역시 나 민주당 여 성 의원 들 은 주디 꾹 다물고있네요 대단해요 
정답:  ['여성/가족'] ,예측:  ['여성/가족', '기타 혐오']
 
문장:  와 이제야 쌍 도 새끼 들 진심 을 알았다 쌍 도 까면 홍어 로 몰아가는게 컨셉 이 아니라 ㄹㅇ 이었구나 니 네 가 홍어 새끼 들 하고 다를게 뭐 냐 또 네 다홍 지랄 할까봐 적눈 다 김대중 , 노무현 , 이석기 , 마오쩌뚱 개 씨발 새끼 
정답:  ['지역', '개인지칭'] ,예측:  ['지역']
 
문장:  무슬림 은 답 없음 그냥 만나지말고 따로 살자 
정답:  ['종교'] ,예측:  ['인종/국적', '종교']
 
문장:  아니 애초 에 염색체 가 다른데 감히 냄져가 여자 가 될라 노 하노 이 와중 에 비 수술 트 렌스 젠더 레즈비언 같운 개똥 같은 소리 하노 
정답:  ['남성', '성소수자'] ,예측:  ['성소수자']
 
문장:  문 재앙 : 이성미 로는 안된다고 !! 손 혜원 빨리 숨겨야해 !!! 생각 없는 기 집 이랑 맘충 들 이나 연 예 기사 에 속지 나머지 놈 들 은 이제 속지 도 않는다고 !! 